# Use lightfm for all positive data and most populat items for cold start recomendations 

In [2]:
import pandas as pd
import numpy as np
import multiprocessing as mp
NUM_CORE = 6 
from lightfm import LightFM

In [3]:
import sys
sys.path.append('..')
from metrics import apk, mapk, precision_at_k, recall_at_k
from utils import get_train_test_data_split, pd_to_scr_matrix
from collections import defaultdict

In [4]:
data = pd.read_csv('../../data/recsys_data.csv')
data['date'] =pd.to_datetime(data['date'])

In [5]:
train, test  = get_train_test_data_split(data, val_start = pd.Timestamp('2020-06-27 00:00:00'), test_days= 1 )

In [6]:
most_popular_items = list(train.loc[train['status']==1, 'survey'].value_counts().index[0:3])

In [7]:
most_popular_items

[100000513, 100000017, 100000040]

In [8]:
test_actual_results = test.loc[ test['status']==1, 
                         :].groupby( ['user'] ).apply(  lambda x: list( x['survey']) ).reset_index()

test_actual_results.columns = ['user', 'true']



In [9]:
test_actual_results

,user,true
0,57141,[100002475]
1,61299,[100000427]
2,64059,"[100006888, 100010236]"
3,65613,"[100006556, 100003291, 100001173]"
4,73206,"[100016937, 100009234]"
...,...,...
25129,62072229,[100001246]
25130,62072244,[100001246]
25131,62072286,[100022773]
25132,62072334,[100005041]


In [10]:
previously_seen_items = train.groupby(['user'])['survey'].apply( lambda x: list(x)).to_dict()

In [11]:
train = train.loc[train['status'] ==1, :]

In [12]:
train_matrix, user_id_cats, survey_id_cats= pd_to_scr_matrix(train)

In [13]:
train_matrix

<38275x13166 sparse matrix of type '<class 'numpy.int64'>'
	with 138362 stored elements in Compressed Sparse Row format>

In [14]:
user_id_cats

CategoricalDtype(categories=[   57141,    61299,    63513,    64407,    65613,    66771,
                     73206,    73368,    73734,    83121,
                  ...
                  62031813, 62031843, 62031858, 62031876, 62031939, 62032119,
                  62032227, 62032293, 62032341, 62032347],
, ordered=True)

In [15]:
survey_id_cats

CategoricalDtype(categories=[100000002, 100000003, 100000004, 100000005, 100000006,
                  100000007, 100000008, 100000009, 100000010, 100000011,
                  ...
                  100042350, 100042351, 100042354, 100042363, 100042364,
                  100042371, 100042374, 100042380, 100042381, 100042425],
, ordered=True)

In [16]:
#use lightfm
model = LightFM(loss='warp')

In [17]:
model.fit(train_matrix, epochs=30)

In [18]:
n_users, n_items = train_matrix.shape

In [19]:
print(n_users, n_items)

38275 13166


In [20]:
def mp_predict_with_seen(data, k=3):
    """
    make predictions using multiprocessing. delete already seen items
    """
    user_id, previous_items = data
    scores = model.predict(user_id, np.arange(n_items))
    top_items = list(survey_id_cats.categories[np.argsort(-scores)])
    top_items =  [item for item in top_items if item not in previous_items]
    return top_items[:k]


In [21]:
# data with user and his previous items 
user_id_previously_seen_items= defaultdict(list)
for i, user in enumerate(user_id_cats.categories):
    user_id_previously_seen_items[i] = previously_seen_items.get(user, []  )
x= list(zip(list( range(n_users )), [user_id_previously_seen_items[i] for i in range(n_users)] ))

In [22]:
# make prediction for all users in train dataset
pool = mp.Pool(NUM_CORE)
list_of_results_seen = pool.map(mp_predict_with_seen, x )
pool.close()
pool.join()

In [23]:
list_of_results_seen[0:5]

[[100000017, 100000019, 100000513],
 [100000071, 100000406, 100000084],
 [100003515, 100000009, 100001142],
 [100001892, 100000017, 100000601],
 [100000408, 100000393, 100000079]]

In [24]:
predictions = pd.DataFrame( user_id_cats.categories )
predictions.columns  = ['user']
predictions['predicted'] = list_of_results_seen

In [25]:
predictions

,user,predicted
0,57141,"[100000017, 100000019, 100000513]"
1,61299,"[100000071, 100000406, 100000084]"
2,63513,"[100003515, 100000009, 100001142]"
3,64407,"[100001892, 100000017, 100000601]"
4,65613,"[100000408, 100000393, 100000079]"
...,...,...
38270,62032119,"[100000335, 100000043, 100000513]"
38271,62032227,"[100000276, 100000601, 100000726]"
38272,62032293,"[100000513, 100000079, 100000017]"
38273,62032341,"[100000017, 100003193, 100000601]"


In [26]:
test_actual_results = pd.merge(test_actual_results, predictions, how= 'left', on = 'user')

In [27]:
test_actual_results

,user,true,predicted
0,57141,[100002475],"[100000017, 100000019, 100000513]"
1,61299,[100000427],"[100000071, 100000406, 100000084]"
2,64059,"[100006888, 100010236]",NaN
3,65613,"[100006556, 100003291, 100001173]","[100000408, 100000393, 100000079]"
4,73206,"[100016937, 100009234]","[100000408, 100000562, 100000482]"
...,...,...,...
25129,62072229,[100001246],NaN
25130,62072244,[100001246],NaN
25131,62072286,[100022773],NaN
25132,62072334,[100005041],NaN


### look at metics for non cold start predictions

In [28]:
only_lfm_mapk = mapk( test_actual_results.loc[ test_actual_results['predicted'].notna() , 'true'], 
                     test_actual_results.loc[  test_actual_results['predicted'].notna() ,'predicted'] , k =3)


only_lfm_precision_at_k = np.mean(test_actual_results.loc[  test_actual_results['predicted'].notna() ,:] .apply( lambda x: precision_at_k(x ['true'],
                                                                                 x['predicted'], k= 3 )  , axis=1))

only_lfm_recall_at_k = np.mean(test_actual_results.loc[  test_actual_results['predicted'].notna() ,:].apply( lambda x: recall_at_k(x ['true'],
                                                                                 x['predicted'], k= 3 )  , axis=1))

In [29]:
print(f'only lightfm map@k = {only_lfm_mapk:.5f} ')
print(f'only lightfm precision@k = {only_lfm_precision_at_k:.5f}')
print(f'only lightfm recall@k = {only_lfm_recall_at_k:.5f}')

only lightfm map@k = 0.00361 
only lightfm precision@k = 0.00390
only lightfm recall@k = 0.00500


In [30]:
test_actual_results['predicted'].isna().mean()

0.37363730405029044

## We have no train data and prediction for 37% of test and will use mostpopular baseline for them

In [31]:
test_actual_results.loc[test_actual_results['predicted'].isna(), 'predicted' ] = test_actual_results.loc[test_actual_results['predicted'].isna(), 'predicted' ].apply(lambda x: x if x==np.nan  else most_popular_items)


In [32]:
lfm_mapk = mapk( test_actual_results['true'], test_actual_results['predicted'] , k =3)
lfm_precision_at_k = np.mean(test_actual_results.apply( lambda x: precision_at_k(x ['true'],
                                                                                 x['predicted'], k= 3 )  , axis=1))

lfm_recall_at_k = np.mean(test_actual_results.apply( lambda x: recall_at_k(x ['true'],
                                                                                 x['predicted'], k= 3 )  , axis=1))

In [33]:
print(f'lightfm map@k = {lfm_mapk:.5f} ')
print(f'lightfm precision@k = {lfm_precision_at_k:.5f}')
print(f'lightfm recall@k = {lfm_recall_at_k:.5f}')

lightfm map@k = 0.00234 
lightfm precision@k = 0.00255
lightfm recall@k = 0.00331


# we significantly improve baseline:
* baseline map@k = 0.00019 
* baseline precision@k = 0.00024
* baseline recall@k = 0.00040